# 00-parsing_antiSMASH



### Overview

Content: 
1. Read in antismash output
2. retreive Regulators seqs from differet families e.g. all LuxR tfs

In [1]:

import sys
import os
def list_of_objects_in_a_dir(dir_path:str): 
    """List all objects in a directory.
    
    Parameters
    ----------
    dir_path : str
        The path to the directory.
        
    Returns
    -------
    list
        A list of the objects in the directory.
    """
    # list to store files
    list_of_files = []

    # Iterate directory
    for path in os.listdir(dir_path):
        # check if current path is a file
        if os.path.isfile(os.path.join(dir_path, path)):
            list_of_files.append(path)
            
    return list_of_files


In [2]:
from teemi.design.fetch_sequences import read_genbank_files
import pandas as pd

In [3]:
go4010 = read_genbank_files('../wet_lab_notebooks/data_for_drylab/Go40110_flye_polish_actinoannotPFAM_antismash7/Go40110_flye_polish_actinoannotPFAM.gbk')[0]

In [4]:
f'Lenght of the genome {len(go4010):,} bp'

'Lenght of the genome 9,637,008 bp'

In [5]:
go4010

SeqRecord(seq=Seq('CAGCTCCGGCACGGAAAGAGGCTGGTCTGGTTTTGTGGGGTGGGGGCTGGGGTT...AGG'), id='.', name='Exported', description='Streptomyces reticuli_0.928 strain strain', dbxrefs=[])

How many clusters

In [6]:
# How many genes do we have?
locus_tags = []
qualifiers = []
cds = []

for feature in go4010.features:
    if feature.type == 'CDS': 
        # getting the locus tag if we see cds
        locus_tags.append(feature.qualifiers['locus_tag'][0])
        qualifiers.append(feature.qualifiers)

        # All CDS features
        cds.append(feature)

        
print('Total coding sequences : ', len(locus_tags))    

Total coding sequences :  8154


In [7]:
cds[:10]

[SeqFeature(SimpleLocation(ExactPosition(202), ExactPosition(1000), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(3087), ExactPosition(3231), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(6166), ExactPosition(7426), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(9178), ExactPosition(9358), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(9504), ExactPosition(9690), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(14019), ExactPosition(14781), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(15280), ExactPosition(15472), strand=-1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(16206), ExactPosition(16326), strand=-1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(16859), ExactPosition(17699), strand=-1), type='CDS', qualifiers=...),
 SeqFeatu

Lets retrieve the locus_tag for all transcriptional regualtors

In [8]:
regulators = []
regulator_descriptions = []
for qualifier in qualifiers: 
    if 'gene_kind' in qualifier:
        if 'regulatory' in qualifier['gene_functions'][0]:
            #print(qualifier['gene_functions'])
            if 'LuxR' in qualifier['gene_functions'][0] or 'SARP' in qualifier['gene_functions'][0]: 
                print(qualifier['gene_functions'][0])
                regulator_descriptions.append(qualifier['gene_functions'][0][30:])
                regulators.append(qualifier)
                
len(regulators)

regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 187.1; E-value: 4.1e-57)
regulatory (smcogs) SMCOG1149:LuxR family transcriptional regulator (Score: 972.3; E-value: 4.8e-294)
regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 166.8; E-value: 6.4e-51)
regulatory (smcogs) SMCOG1041:transcriptional regulator, SARP family (Score: 314.4; E-value: 3.3e-95)
regulatory (smcogs) SMCOG1041:transcriptional regulator, SARP family (Score: 259.2; E-value: 1.5e-78)
regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 66; E-value: 4.5e-20)
regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 143.4; E-value: 9.7e-44)
regulatory (smcogs) SMCOG1041:transcriptional regulator, SARP family (Score: 266.8; E-value: 7.8e-81)
regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 151.1; E-value: 4.3e-46)
regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response re

18

In [9]:
regulator_descriptions

['LuxR family DNA-binding response regulator (Score: 187.1; E-value: 4.1e-57)',
 'LuxR family transcriptional regulator (Score: 972.3; E-value: 4.8e-294)',
 'LuxR family DNA-binding response regulator (Score: 166.8; E-value: 6.4e-51)',
 'transcriptional regulator, SARP family (Score: 314.4; E-value: 3.3e-95)',
 'transcriptional regulator, SARP family (Score: 259.2; E-value: 1.5e-78)',
 'LuxR family DNA-binding response regulator (Score: 66; E-value: 4.5e-20)',
 'LuxR family DNA-binding response regulator (Score: 143.4; E-value: 9.7e-44)',
 'transcriptional regulator, SARP family (Score: 266.8; E-value: 7.8e-81)',
 'LuxR family DNA-binding response regulator (Score: 151.1; E-value: 4.3e-46)',
 'LuxR family DNA-binding response regulator (Score: 175.3; E-value: 1.7e-53)',
 'transcriptional regulator, SARP family (Score: 252.9; E-value: 1.2e-76)',
 'LuxR family DNA-binding response regulator (Score: 84.7; E-value: 8.6e-26)',
 'transcriptional regulator, SARP family (Score: 511.2; E-value:

In [10]:
locus_tags_to_search_for = []
for qualifier in regulators: 
    locus_tags_to_search_for.append(qualifier['locus_tag'][0])

In [11]:
print(f'Total sequences within these two regulator families: {len(locus_tags_to_search_for)}')
locus_tags_to_search_for

Total sequences within these two regulator families: 18


['LLPMBPKK_00292',
 'LLPMBPKK_00328',
 'LLPMBPKK_00586',
 'LLPMBPKK_01488',
 'LLPMBPKK_02197',
 'LLPMBPKK_02209',
 'LLPMBPKK_02563',
 'LLPMBPKK_02633',
 'LLPMBPKK_02635',
 'LLPMBPKK_02637',
 'LLPMBPKK_02662',
 'LLPMBPKK_05992',
 'LLPMBPKK_06337',
 'LLPMBPKK_06422',
 'LLPMBPKK_06907',
 'LLPMBPKK_07531',
 'LLPMBPKK_07744',
 'LLPMBPKK_07949']

### Manual curation: 
-  LuxR 11
- manual SARP 8

### In which gene clusters are they found?

In [12]:

regions_names = list_of_objects_in_a_dir('../wet_lab_notebooks/data_for_drylab/Go40110_flye_polish_actinoannotPFAM_antismash7/')

genbank_regions = []
for gb in regions_names: 
    if gb.endswith('.gbk') and gb.startswith('..'): 
        genbank_regions.append(gb)

genbank_regions_sorted = sorted(genbank_regions)



In [13]:
# lets parse: 
all_genbanks = []
for region in genbank_regions_sorted: 
    all_genbanks.append( read_genbank_files(f'../wet_lab_notebooks/data_for_drylab/Go40110_flye_polish_actinoannotPFAM_antismash7/{region}')[0])

In [14]:
regions_for_each_locus_tag = []
for lc_tag in locus_tags_to_search_for: 
    for i in range(len(all_genbanks)): 
       for feature in all_genbanks[i].features:
            if feature.type == "CDS":
                if feature.qualifiers['locus_tag'][0] == lc_tag: 
                    # getting the locus tag if we see cds
                    locus_tag_and_region = {'locus_tag':lc_tag, 'region': f'region {i+1}'}
                    regions_for_each_locus_tag.append(locus_tag_and_region)
regions_df = pd.DataFrame(regions_for_each_locus_tag, columns =['locus_tag', 'region'])
regions_df['regulator_number'] = [f'Reg {i+1}' for i in range(len(regions_df))]

regions_df['description'] = regulator_descriptions
regions_df

,locus_tag,region,regulator_number,description
0,LLPMBPKK_00292,region 1,Reg 1,LuxR family DNA-binding response regulator (Sc...
1,LLPMBPKK_00328,region 1,Reg 2,LuxR family transcriptional regulator (Score: ...
2,LLPMBPKK_00586,region 4,Reg 3,LuxR family DNA-binding response regulator (Sc...
3,LLPMBPKK_01488,region 10,Reg 4,"transcriptional regulator, SARP family (Score:..."
4,LLPMBPKK_02197,region 12,Reg 5,"transcriptional regulator, SARP family (Score:..."
5,LLPMBPKK_02209,region 12,Reg 6,LuxR family DNA-binding response regulator (Sc...
6,LLPMBPKK_02563,region 13,Reg 7,LuxR family DNA-binding response regulator (Sc...
7,LLPMBPKK_02633,region 14,Reg 8,"transcriptional regulator, SARP family (Score:..."
8,LLPMBPKK_02635,region 14,Reg 9,LuxR family DNA-binding response regulator (Sc...
9,LLPMBPKK_02637,region 14,Reg 10,LuxR family DNA-binding response regulator (Sc...


Manually, we have inspected the clusters and we are interested in the products from  14, 28, 38

### All regulators

In [15]:
## If we just look at all regulators
transcriptional_regulators = []
for qualifier in qualifiers: 
    if 'product' in qualifier: 
        if 'transcriptional regulator' in  qualifier['product'][0]: 
            #tf = [qualifier['locus_tag'],qualifier['product']]
            transcriptional_regulators.append(qualifier['locus_tag'][0])
print(f'Total transcriptional regulators : {len(transcriptional_regulators)}')
transcriptional_regulators[:10]

Total transcriptional regulators : 531


['LLPMBPKK_00020',
 'LLPMBPKK_00064',
 'LLPMBPKK_00191',
 'LLPMBPKK_00217',
 'LLPMBPKK_00244',
 'LLPMBPKK_00260',
 'LLPMBPKK_00283',
 'LLPMBPKK_00293',
 'LLPMBPKK_00328',
 'LLPMBPKK_00353']

In [16]:
regulators_in_all_bgcs = []
regulator_descriptions_in_all_bgcs = {}
for qualifier in qualifiers:
    if 'gene_kind' in qualifier:
        locus_tag = qualifier['locus_tag'][0]
        if 'regulatory' in qualifier['gene_functions'][0]:
            regulator_descriptions_in_all_bgcs[locus_tag] = qualifier['gene_functions'][0][30:]
            regulators_in_all_bgcs.append(qualifier)
        

print(f"Number of regulators: {len(regulators_in_all_bgcs)}")
print("Sample of regulator_descriptions:")
regulator_descriptions_in_all_bgcs

Number of regulators: 87
Sample of regulator_descriptions:


{'LLPMBPKK_00291': 'sensor histidine kinase (Score: 173.2; E-value: 1.5e-52)',
 'LLPMBPKK_00292': 'LuxR family DNA-binding response regulator (Score: 187.1; E-value: 4.1e-57)',
 'LLPMBPKK_00293': 'MarR family transcriptional regulator (Score: 104.6; E-value: 4.9e-32)',
 'LLPMBPKK_00297': 'response regulator (Score: 163.1; E-value: 1.2e-49)',
 'LLPMBPKK_00328': 'LuxR family transcriptional regulator (Score: 972.3; E-value: 4.8e-294)',
 'LLPMBPKK_00474': 'MarR family transcriptional regulator (Score: 59.4; E-value: 4.7e-18)',
 'LLPMBPKK_00476': 'TetR family transcriptional regulator (Score: 90.6; E-value: 2.7e-27)',
 'LLPMBPKK_00481': 'TetR family transcriptional regulator (Score: 88.7; E-value: 1.1e-26)',
 'LLPMBPKK_00488': 'RNA polymerase, sigma-24 subunit, ECF subfamily (Score: 121.3; E-value: 5.4e-37)',
 'LLPMBPKK_00524': 'transcriptional regulator, MerR family (Score: 110.7; E-value: 1.9e-33)',
 'LLPMBPKK_00577': 'LysR family transcriptional regulator (Score: 158.7; E-value: 3e-48)'

Find the regions

In [17]:
regions_for_each_locus_tag = []
for lc_tag in regulator_descriptions_in_all_bgcs: 
    for i in range(len(all_genbanks)): 
       for feature in all_genbanks[i].features:
            if feature.type == "CDS":
                if feature.qualifiers['locus_tag'][0] == lc_tag: 
                    # getting the locus tag if we see cds
                    locus_tag_and_region = {'locus_tag':lc_tag, 'region': f'region {i+1}'}
                    regions_for_each_locus_tag.append(locus_tag_and_region)

regions_df1 = pd.DataFrame(regions_for_each_locus_tag, columns =['locus_tag', 'region'])

# Add descriptions from the regulator_descriptions dictionary
descriptions_for_bgc_rgs = []
for description in regions_df1['locus_tag'].to_list(): 
    my_description = regulator_descriptions_in_all_bgcs[description]
    descriptions_for_bgc_rgs.append(my_description)
regions_df1['description'] =descriptions_for_bgc_rgs

regions_df1.to_excel('data_for_drylab/all_regulators_in_bgcs.xlsx')

lets rows with regulators we already have made:

In [18]:
# Now, remove rows that have locus_tag values present in regions_df
regions_df1_filtered = regions_df1[~regions_df1['locus_tag'].isin(regions_df['locus_tag'])]
regions_df1_filtered.to_excel('data_for_drylab/all_regulators_in_bgcs_filtered.xlsx')

# Display the result
print(f"Number of rows in regions_df1 after removal: {len(regions_df1_filtered)}")
regions_df1_filtered

Number of rows in regions_df1 after removal: 69


,locus_tag,region,description
0,LLPMBPKK_00291,region 1,sensor histidine kinase (Score: 173.2; E-value...
2,LLPMBPKK_00293,region 1,MarR family transcriptional regulator (Score: ...
3,LLPMBPKK_00297,region 1,response regulator (Score: 163.1; E-value: 1.2...
5,LLPMBPKK_00474,region 2,MarR family transcriptional regulator (Score: ...
6,LLPMBPKK_00476,region 2,TetR family transcriptional regulator (Score: ...
...,...,...,...
81,LLPMBPKK_07870,region 37,TetR family transcriptional regulator (Score: ...
82,LLPMBPKK_07886,region 37,"transcriptional regulator, MarR family (Score:..."
83,LLPMBPKK_07922,region 38,TetR family transcriptional regulator (Score: ...
85,LLPMBPKK_07960,region 38,TetR family transcriptional regulator (Score: ...


### We focus on BGC 14

In [19]:
# Define the regions to exclude
regions_to_keep = ['region 14']

# Filter the DataFrame
filtered_df_14 = regions_df1_filtered[regions_df1_filtered['region'].isin(regions_to_keep)]

# Optionally, reset the index
filtered_df_14 = filtered_df_14.reset_index(drop=True)

# excel file
filtered_df_14.to_excel('data_for_drylab/region14_regulators_in_bgcs_filtered.xlsx')

# Display the filtered DataFrame
filtered_df_14

,locus_tag,region,description
0,LLPMBPKK_02636,region 14,sensor histidine kinase (Score: 155.7; E-value...
1,LLPMBPKK_02638,region 14,sensor histidine kinase (Score: 185.3; E-value...
2,LLPMBPKK_02661,region 14,LysR family transcriptional regulator (Score: ...
3,LLPMBPKK_02680,region 14,AraC family transcriptional regulator (Score: ...
4,LLPMBPKK_02681,region 14,"transcriptional regulator, MerR family (Score:..."


Lets get the sequences: 

In [20]:
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation

In [21]:
my_regulator_sequences_for_bgc14 = []
for i in range(len(cds)): 
    if cds[i].qualifiers['locus_tag'][0] in filtered_df_14['locus_tag'].to_list(): 
        feature_sequence = SeqRecord(cds[i].extract(go4010.seq), id = cds[i].qualifiers['locus_tag'][0],name = cds[i].qualifiers['locus_tag'][0], description = cds[i].qualifiers['gene_functions'][0] )
        
        # annotation
        feature_location = FeatureLocation(start=0, end=len(feature_sequence))
        my_feature = SeqFeature(location=feature_location, type='gene', 
                                qualifiers={"name": cds[i].qualifiers['locus_tag'][0], 
                                            "product": cds[i].qualifiers['locus_tag'][0],
                                            "description": cds[i].qualifiers['gene_functions'][0]})
        feature_sequence.features.append(my_feature)
        
        # saving feature
        my_regulator_sequences_for_bgc14.append(feature_sequence)

my_regulator_sequences_for_bgc14

[SeqRecord(seq=Seq('ATGTGCCGACACAAGTCCTCGCCAAAGGCGACCGAGGGCTCCGGGCAGCCGGAC...TGA'), id='LLPMBPKK_02636', name='LLPMBPKK_02636', description='regulatory (smcogs) SMCOG1048:sensor histidine kinase (Score: 155.7; E-value: 3.2e-47)', dbxrefs=[]),
 SeqRecord(seq=Seq('GTGTCCCAGTGGCAGCAGCAGCCGAGCGACCGGCTCACCCTGGACCGGATCCGT...TGA'), id='LLPMBPKK_02638', name='LLPMBPKK_02638', description='regulatory (smcogs) SMCOG1048:sensor histidine kinase (Score: 185.3; E-value: 3.3e-56)', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGACATCGAATTGAGGCATGCGCGGATCGTCGTCACGATAAGTGAAGCCGGA...TGA'), id='LLPMBPKK_02661', name='LLPMBPKK_02661', description='regulatory (smcogs) SMCOG1014:LysR family transcriptional regulator (Score: 104.2; E-value: 1.3e-31)', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGGGGAACAGGCGTCGTGGACGCGGACCCGGCTGGGCCGTTGCGGGCCGCCG...TGA'), id='LLPMBPKK_02680', name='LLPMBPKK_02680', description='regulatory (smcogs) SMCOG1260:AraC family transcriptional regulator (Score: 79.9; E-value: 4e-24)', dbxrefs=[]),
 Se

In [22]:
with open('../wet_lab_notebooks/data_for_drylab/remaining_regulators_from_region14.gb', 'w') as outfile:
    for i in range(0, len(my_regulator_sequences_for_bgc14)):
        my_regulator_sequences_for_bgc14[i].annotations["molecule_type"] = "DNA"
        print(my_regulator_sequences_for_bgc14[i].format("gb"), file = outfile)

In [23]:
f'Number of regulator sequences: {len(my_regulator_sequences_for_bgc14)}'

'Number of regulator sequences: 5'

## Checking colibormycin - Streptomyces sp. CS147


- DBP21_RS21605 cbm16 SARP transcriptional regulator 4,741,630 - 4,742,427

- DBP21_RS21610 cbm17 LysR transcriptional regulator 4,742,644 - 4,743,618

https://www.ncbi.nlm.nih.gov/nuccore/NZ_QBIA00000000.1

In [24]:
s_cs147 = read_genbank_files('../wet_lab_notebooks/s_cs147.gb')[0]

In [25]:
s_cs147

SeqRecord(seq=Seq('AGTTGAGACTGCCGCCACTGCTTCTCCGGACGACCAGTGGAAAAGGCGTCTGCG...GGC'), id='KZ819182.1', name='KZ819182', description='Streptomyces sp. CS147 genomic scaffold scaffold00001, whole genome shotgun sequence', dbxrefs=['BioProject:PRJNA449329', 'BioSample:SAMN08896652'])

In [26]:
tags_from_paper = ['DBP21_21605', 'DBP21_21610']

new_tags_from_my_search = ['DBP21_21620', 'DBP21_21625']
# How many genes do we have?
locus_tags_s_cs147 = []
qualifiers_s_cs147 = []
cds_s_cs147 = []

for feature in s_cs147.features:
    if feature.type == 'CDS': 
        # getting the locus tag if we see cds

        #print(feature.qualifiers['locus_tag'][0])
        if feature.qualifiers['locus_tag'][0] in new_tags_from_my_search:
            locus_tags_s_cs147.append(feature.qualifiers['locus_tag'][0])
            qualifiers_s_cs147.append(feature.qualifiers)

            # All CDS features
            cds_s_cs147.append(feature)

In [27]:
qualifiers_s_cs147

[{'locus_tag': ['DBP21_21620'],
  'inference': ['COORDINATES: similar to AA sequence:RefSeq:WP_015100637.1'],
  'note': ['Derived by automated computational analysis using gene prediction method: Protein Homology.'],
  'codon_start': ['1'],
  'transl_table': ['11'],
  'product': ['SARP family transcriptional regulator'],
  'protein_id': ['PVC99858.1'],
  'translation': ['MEINILGALSLTCGQRSAEVASKRVSTVLTLLALSPGTPIPFDQLVDELWADQPMGNARNALQANVSRLRRLLESVTRTPSDGIVRTVSSGYLLDVPTEAVDVHRFVALASAGAGLVHKDPAEAVEKLERALGLWRGPALMDARDGLRCRVQAAHLEERRLTVYEDLVAAKLAACSERISLPELRQLAADHSERERLTELLMLALYREGRQAEALDVFHGARRRLAAELGLEPGPALHLVYQAILNQDPMLGQPRHVLLYAAPAS']},
 {'locus_tag': ['DBP21_21625'],
  'inference': ['COORDINATES: similar to AA sequence:RefSeq:WP_015037412.1'],
  'note': ['Derived by automated computational analysis using gene prediction method: Protein Homology.'],
  'codon_start': ['1'],
  'transl_table': ['11'],
  'product': ['LysR family transcriptional regulator'],
  'protein_id': ['PVC99859.1'],
 

lets get the sequences

In [28]:
regulator_sequences_from_paper = []
for i in range(len(cds_s_cs147)): 
    if cds_s_cs147[i].qualifiers['locus_tag'][0] in new_tags_from_my_search: 
        feature_sequence = SeqRecord(cds_s_cs147[i].extract(s_cs147.seq), id = cds_s_cs147[i].qualifiers['locus_tag'][0],name = cds_s_cs147[i].qualifiers['locus_tag'][0] )
        
        # annotation
        feature_location = FeatureLocation(start=0, end=len(feature_sequence))
        my_feature = SeqFeature(location=feature_location, type='gene', 
                                qualifiers={"name": cds[i].qualifiers['locus_tag'][0], 
                                            "product": cds[i].qualifiers['locus_tag'][0],
                                            })
        feature_sequence.features.append(my_feature)
        
        # saving feature
        regulator_sequences_from_paper.append(feature_sequence)
    
regulator_sequences_from_paper

[SeqRecord(seq=Seq('GTGGAAATCAATATTCTGGGTGCGCTCTCGCTGACGTGTGGTCAGCGGTCGGCC...TGA'), id='DBP21_21620', name='DBP21_21620', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGATCTCGAACTACGACATGCCCGAATTGTGCTGACAATAGATGCCTGTGGC...TGA'), id='DBP21_21625', name='DBP21_21625', description='<unknown description>', dbxrefs=[])]

In [29]:
regulator_sequences_from_paper[0].name = 'SARP (DBP21_21620)'
print(len(regulator_sequences_from_paper[0]))#.name = 'SARP predicted from paper'

regulator_sequences_from_paper[1].name = 'LysR (DBP21_21625)'
print(len(regulator_sequences_from_paper[1]))#.name = 'SARP predicted from paper'


798
975


In [30]:
regulator_sequences_from_paper

[SeqRecord(seq=Seq('GTGGAAATCAATATTCTGGGTGCGCTCTCGCTGACGTGTGGTCAGCGGTCGGCC...TGA'), id='DBP21_21620', name='SARP (DBP21_21620)', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGATCTCGAACTACGACATGCCCGAATTGTGCTGACAATAGATGCCTGTGGC...TGA'), id='DBP21_21625', name='LysR (DBP21_21625)', description='<unknown description>', dbxrefs=[])]

In [31]:
regulators = read_genbank_files('../wet_lab_notebooks/data_for_drylab/GOE_regulators.gb')

sarp_11 = regulators[10]
lysR_bgc_14 = my_regulator_sequences_for_bgc14[2]


In [32]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

alignments_SARPs = pairwise2.align.globalxx(regulator_sequences_from_paper[0].translate(), sarp_11.translate())

print(format_alignment(*alignments_SARPs[0]))

ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 0
/molecule_type=protein
Seq('VEIN--I--LGALSLTC-----GQ-RS--AEVA-SKRVSTVLT--L-L-ALSPG-TPIPF-DQLV-DELWADQP---...AS--*')
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||      |  ||  |        |  |   | |  |||    ||  | | ||||| | ||  |||  |||||      |||    |||
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 0
/molecule_type=protein
Seq('----MDIQVLG--S---VSVRYG-NR-QHA-V-GSKR----LTAILALLALSPGRT-IP-ADQL-EDELWA---GRT...--HR*')
  Score=160



/Users/lucaslevassor/opt/anaconda3/envs/streptocad/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [33]:
from Bio import pairwise2
alignments_LysR = pairwise2.align.globalxx(regulator_sequences_from_paper[1].translate(), lysR_bgc_14.translate())
print(format_alignment(*alignments_LysR[0]))


ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 0
/molecule_type=protein
Seq('MDL-ELRHARIVL-TID--ACGSISRA-AVRLN-L--PQPSLTAQLRRIEKSV---GGD-LFI-RSR...RP--*')
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||  ||||||||  ||   | |||||  |   | |  |||||||||||||      ||  ||  ||||||    |||
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 0
/molecule_type=protein
Seq('MD-IELRHARIV-VTI-SEA-GSISR-GA---NELSLPQPSLTAQLRRIE---RALGG-ELF-ERSR...--AS*')
  Score=170



### Which regulator is closest homolog to the SARP from the paper? 

In [34]:
for reg in regulators: 
    score = pairwise2.align.globalxx(regulator_sequences_from_paper[0].translate(), reg.translate(), score_only=True)
    percent_identity = score/len(regulator_sequences_from_paper[0].translate())

    print(f'Score: {score}; percent identity: {percent_identity:.2f} ; for {reg.name} against the paper {regulator_sequences_from_paper[0].name}')

Score: 148.0; percent identity: 0.56 ; for LLPMBPKK_00292 against the paper SARP (DBP21_21620)
Score: 148.0; percent identity: 0.56 ; for LLPMBPKK_00328 against the paper SARP (DBP21_21620)
Score: 149.0; percent identity: 0.56 ; for LLPMBPKK_00586 against the paper SARP (DBP21_21620)
Score: 147.0; percent identity: 0.55 ; for LLPMBPKK_01488 against the paper SARP (DBP21_21620)
Score: 154.0; percent identity: 0.58 ; for LLPMBPKK_02197 against the paper SARP (DBP21_21620)
Score: 152.0; percent identity: 0.57 ; for LLPMBPKK_02209 against the paper SARP (DBP21_21620)
Score: 149.0; percent identity: 0.56 ; for LLPMBPKK_02563 against the paper SARP (DBP21_21620)
Score: 151.0; percent identity: 0.57 ; for LLPMBPKK_02633 against the paper SARP (DBP21_21620)
Score: 150.0; percent identity: 0.56 ; for LLPMBPKK_02635 against the paper SARP (DBP21_21620)
Score: 149.0; percent identity: 0.56 ; for LLPMBPKK_02637 against the paper SARP (DBP21_21620)
Score: 160.0; percent identity: 0.60 ; for LLPMBPK

In [35]:
for reg in my_regulator_sequences_for_bgc14: 
    score = pairwise2.align.globalxx(regulator_sequences_from_paper[1].translate(), reg.translate(), score_only=True)
    percent_identity = score/len(regulator_sequences_from_paper[1].translate())
    print(f'Score: {score}; percent identity: {percent_identity:.2f} ; for {reg.name} against the paper {regulator_sequences_from_paper[1].name}')

Score: 150.0; percent identity: 0.46 ; for LLPMBPKK_02636 against the paper LysR (DBP21_21625)
Score: 148.0; percent identity: 0.46 ; for LLPMBPKK_02638 against the paper LysR (DBP21_21625)
Score: 170.0; percent identity: 0.52 ; for LLPMBPKK_02661 against the paper LysR (DBP21_21625)
Score: 149.0; percent identity: 0.46 ; for LLPMBPKK_02680 against the paper LysR (DBP21_21625)
Score: 151.0; percent identity: 0.46 ; for LLPMBPKK_02681 against the paper LysR (DBP21_21625)


In [36]:
# FInd all SARPs in the 

In [37]:
regions_df1

,locus_tag,region,description
0,LLPMBPKK_00291,region 1,sensor histidine kinase (Score: 173.2; E-value...
1,LLPMBPKK_00292,region 1,LuxR family DNA-binding response regulator (Sc...
2,LLPMBPKK_00293,region 1,MarR family transcriptional regulator (Score: ...
3,LLPMBPKK_00297,region 1,response regulator (Score: 163.1; E-value: 1.2...
4,LLPMBPKK_00328,region 1,LuxR family transcriptional regulator (Score: ...
...,...,...,...
82,LLPMBPKK_07886,region 37,"transcriptional regulator, MarR family (Score:..."
83,LLPMBPKK_07922,region 38,TetR family transcriptional regulator (Score: ...
84,LLPMBPKK_07949,region 38,"transcriptional regulator, SARP family (Score:..."
85,LLPMBPKK_07960,region 38,TetR family transcriptional regulator (Score: ...


In [38]:
# Define the regions to exclude
regions_to_keep = ['region 14']

# Filter the DataFrame
df_14 = regions_df1[regions_df1['region'].isin(regions_to_keep)]
df_14_list_of_locus_tags = df_14['locus_tag'].to_list()
df_14

,locus_tag,region,description
39,LLPMBPKK_02633,region 14,"transcriptional regulator, SARP family (Score:..."
40,LLPMBPKK_02635,region 14,LuxR family DNA-binding response regulator (Sc...
41,LLPMBPKK_02636,region 14,sensor histidine kinase (Score: 155.7; E-value...
42,LLPMBPKK_02637,region 14,LuxR family DNA-binding response regulator (Sc...
43,LLPMBPKK_02638,region 14,sensor histidine kinase (Score: 185.3; E-value...
44,LLPMBPKK_02661,region 14,LysR family transcriptional regulator (Score: ...
45,LLPMBPKK_02662,region 14,"transcriptional regulator, SARP family (Score:..."
46,LLPMBPKK_02680,region 14,AraC family transcriptional regulator (Score: ...
47,LLPMBPKK_02681,region 14,"transcriptional regulator, MerR family (Score:..."


In [39]:
df_14_list_of_locus_tags

['LLPMBPKK_02633',
 'LLPMBPKK_02635',
 'LLPMBPKK_02636',
 'LLPMBPKK_02637',
 'LLPMBPKK_02638',
 'LLPMBPKK_02661',
 'LLPMBPKK_02662',
 'LLPMBPKK_02680',
 'LLPMBPKK_02681']

In [40]:
bgc14_rgs = []
for i in range(len(cds)): 
    if cds[i].qualifiers['locus_tag'][0] in df_14['locus_tag'].to_list(): 
        feature_sequence = SeqRecord(cds[i].extract(go4010.seq), id = cds[i].qualifiers['locus_tag'][0],name = cds[i].qualifiers['locus_tag'][0], description = cds[i].qualifiers['gene_functions'][0] )
        
        # annotation
        feature_location = FeatureLocation(start=0, end=len(feature_sequence))
        my_feature = SeqFeature(location=feature_location, type='gene', 
                                qualifiers={"name": cds[i].qualifiers['locus_tag'][0], 
                                            "product": cds[i].qualifiers['locus_tag'][0],
                                            "description": cds[i].qualifiers['gene_functions'][0]})
        feature_sequence.features.append(my_feature)
        
        # saving feature
        bgc14_rgs.append(feature_sequence)

bgc14_rgs

[SeqRecord(seq=Seq('ATGGCGCATCTGGACATCGGGCTCCTGGGACCGTTCACGGCGTGCATCGACGGT...TAG'), id='LLPMBPKK_02633', name='LLPMBPKK_02633', description='regulatory (smcogs) SMCOG1041:transcriptional regulator, SARP family (Score: 266.8; E-value: 7.8e-81)', dbxrefs=[]),
 SeqRecord(seq=Seq('GTGATTCGGATCCTGCTCGCAGAAGACATGAACATGGTCCGCGGGGCGCTGGTC...TAG'), id='LLPMBPKK_02635', name='LLPMBPKK_02635', description='regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 151.1; E-value: 4.3e-46)', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGTGCCGACACAAGTCCTCGCCAAAGGCGACCGAGGGCTCCGGGCAGCCGGAC...TGA'), id='LLPMBPKK_02636', name='LLPMBPKK_02636', description='regulatory (smcogs) SMCOG1048:sensor histidine kinase (Score: 155.7; E-value: 3.2e-47)', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGACCGACACCGACAACGACCGGATCCGCGTGGTCGTGGTGGACGACCACGCC...TGA'), id='LLPMBPKK_02637', name='LLPMBPKK_02637', description='regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 175.3; E-value:

In [41]:
# Pairwise alignment of SARP

In [42]:
# Initialize an empty list to store each row's data
data = []

# Iterate through each regulator
for reg in bgc14_rgs:
    # Perform the alignment and calculate the score
    translated_paper_seq = regulator_sequences_from_paper[0].translate()
    translated_reg_seq = reg.translate()
    score = pairwise2.align.globalxx(translated_paper_seq, translated_reg_seq, score_only=True)
    
    # Calculate the percent identity
    percent_identity = round(score / len(translated_paper_seq),2)
    
    # Append the results as a dictionary to the data list
    data.append({
        'Score': score,
        'Percent Identity': percent_identity,
        'Regulator Name': reg.name,
        'Paper Regulator Name': regulator_sequences_from_paper[0].name
    })

# Create a DataFrame from the collected data
df = pd.DataFrame(data)
df.to_clipboard()

df

,Score,Percent Identity,Regulator Name,Paper Regulator Name
0,151.0,0.57,LLPMBPKK_02633,SARP (DBP21_21620)
1,150.0,0.56,LLPMBPKK_02635,SARP (DBP21_21620)
2,148.0,0.56,LLPMBPKK_02636,SARP (DBP21_21620)
3,149.0,0.56,LLPMBPKK_02637,SARP (DBP21_21620)
4,149.0,0.56,LLPMBPKK_02638,SARP (DBP21_21620)
5,149.0,0.56,LLPMBPKK_02661,SARP (DBP21_21620)
6,160.0,0.60,LLPMBPKK_02662,SARP (DBP21_21620)
7,147.0,0.55,LLPMBPKK_02680,SARP (DBP21_21620)
8,148.0,0.56,LLPMBPKK_02681,SARP (DBP21_21620)


In [43]:
# Initialize an empty list to store each row's data
data = []

# Iterate through each regulator
for reg in bgc14_rgs:
    # Perform the alignment and calculate the score
    translated_paper_seq = regulator_sequences_from_paper[1].translate()
    translated_reg_seq = reg.translate()
    score = pairwise2.align.globalxx(translated_paper_seq, translated_reg_seq, score_only=True)
    
    # Calculate the percent identity
    percent_identity = round(score / len(translated_paper_seq), 2)     
    # Append the results as a dictionary to the data list
    data.append({
        'Score': score,
        'Percent Identity': percent_identity,
        'Regulator Name': reg.name,
        'Paper Regulator Name': regulator_sequences_from_paper[1].name
    })

# Create a DataFrame from the collected data
df = pd.DataFrame(data)
df.to_clipboard()
df

,Score,Percent Identity,Regulator Name,Paper Regulator Name
0,152.0,0.47,LLPMBPKK_02633,LysR (DBP21_21625)
1,146.0,0.45,LLPMBPKK_02635,LysR (DBP21_21625)
2,150.0,0.46,LLPMBPKK_02636,LysR (DBP21_21625)
3,153.0,0.47,LLPMBPKK_02637,LysR (DBP21_21625)
4,148.0,0.46,LLPMBPKK_02638,LysR (DBP21_21625)
5,170.0,0.52,LLPMBPKK_02661,LysR (DBP21_21625)
6,153.0,0.47,LLPMBPKK_02662,LysR (DBP21_21625)
7,149.0,0.46,LLPMBPKK_02680,LysR (DBP21_21625)
8,151.0,0.46,LLPMBPKK_02681,LysR (DBP21_21625)


In [44]:
bgc14_rgs_sarp = [
    SeqRecord(seq=reg.seq.translate(), id=reg.name, description="")
    for reg in [regulator_sequences_from_paper[0]] + bgc14_rgs
]
bgc14_rgs_sarp

[SeqRecord(seq=Seq('VEINILGALSLTCGQRSAEVASKRVSTVLTLLALSPGTPIPFDQLVDELWADQP...AS*'), id='SARP (DBP21_21620)', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MAHLDIGLLGPFTACIDGASIVPTAAKQRRLLALLAIDTGHEVSMASIAEGLWE...PL*'), id='LLPMBPKK_02633', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('VIRILLAEDMNMVRGALVALLSLENDFEVVSELERGDEILDAALKAKPDIAVID...LV*'), id='LLPMBPKK_02635', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MCRHKSSPKATEGSGQPDALLVRSGNLPAPRLGRLVLLAALLCYFSITLLNILS...AA*'), id='LLPMBPKK_02636', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MTDTDNDRIRVVVVDDHAVMRAGVVALLTGEPTIEIVGEASDGREGVDLVRRLA...RS*'), id='LLPMBPKK_02637', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('VSQWQQQPSDRLTLDRIRRWNLAGWILLGLLPTAVAVADRSPARKFWLLGLLAL...QG*'), id='LLPMBPKK_02638', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MDIELRHARIVVTISEAGSISRGANELSLPQPSL

In [45]:
bgc14_rgs_lysR = [
    SeqRecord(seq=reg.seq.translate(), id=reg.name, description="")
    for reg in [regulator_sequences_from_paper[1]] + bgc14_rgs
]

bgc14_rgs_lysR

[SeqRecord(seq=Seq('MDLELRHARIVLTIDACGSISRAAVRLNLPQPSLTAQLRRIEKSVGGDLFIRSR...RP*'), id='LysR (DBP21_21625)', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MAHLDIGLLGPFTACIDGASIVPTAAKQRRLLALLAIDTGHEVSMASIAEGLWE...PL*'), id='LLPMBPKK_02633', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('VIRILLAEDMNMVRGALVALLSLENDFEVVSELERGDEILDAALKAKPDIAVID...LV*'), id='LLPMBPKK_02635', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MCRHKSSPKATEGSGQPDALLVRSGNLPAPRLGRLVLLAALLCYFSITLLNILS...AA*'), id='LLPMBPKK_02636', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MTDTDNDRIRVVVVDDHAVMRAGVVALLTGEPTIEIVGEASDGREGVDLVRRLA...RS*'), id='LLPMBPKK_02637', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('VSQWQQQPSDRLTLDRIRRWNLAGWILLGLLPTAVAVADRSPARKFWLLGLLAL...QG*'), id='LLPMBPKK_02638', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MDIELRHARIVVTISEAGSISRGANELSLPQPSL

In [46]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from io import StringIO
# Write records to a FASTA file
# with open("regulators_region14_SARP.fasta", "w") as output_handle:
#     SeqIO.write(bgc14_rgs_sarp, output_handle, "fasta")


# with open("regulators_region14_LysR.fasta", "w") as output_handle:
#     SeqIO.write(bgc14_rgs_lysR, output_handle, "fasta")
    